<b style="font-size:34px; line-height:1.2"><center>SQL</b></center>

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

<b style="font-size:20px; line-height:1.2"><center>Структура данных</b></center>

Таблица **books** (содержит данные о книгах):

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица **authors** (содержит данные об авторах):

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица **publishers** (содержит данные об издательствах):

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица **ratings** (содержит данные о пользовательских оценках книг):

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица **reviews** (содержит данные о пользовательских обзорах на книги):

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<b style="font-size:20px; line-height:1.2"><center>SQL-запрос</b></center>

## 1. Первые строки каждой таблицы:

 1. *books*

In [2]:
query = ''' 
    SELECT *
    FROM
        books
    '''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


 2. *authors*

In [3]:
query = ''' 
    SELECT *
    FROM
        authors
    LIMIT 3
    '''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


 3. *publishers*

In [4]:
query = ''' 
    SELECT *
    FROM
        publishers
    LIMIT 3
    '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


 4. *ratings*

In [5]:
query = ''' 
    SELECT *
    FROM
        ratings
    LIMIT 3
    '''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


 5. *reviews*

In [6]:
query = ''' 
    SELECT *
    FROM
        reviews
    LIMIT 3
    '''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


## 2. Посчитаем сколько книг вышло после 1 января 2000 года:

Выполняем запрос и сохраняем результат выполнения в DataFrame:

In [7]:
query = ''' 
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date >= '2000-01-01'
        '''
count_books = pd.io.sql.read_sql(query, con = engine)
print('Количество книг вышло после 1 января 2000 года:', count_books['count'][0])

Количество книг вышло после 1 января 2000 года: 821


## 3. Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [8]:
query = ''' 
        SELECT
          book_id as "ID книги", 
          title as "Название книги",
          ROUND(t_mean_ratings.avg_rating, 2) as "Оценка",
          t_cnt_review.cnt_review as "Количество обзоров"
        FROM
            books
            LEFT JOIN (
                SELECT
                    AVG(rating) as avg_rating,
                    book_id as name_id
                FROM
                    ratings
                GROUP BY
                    book_id
            ) as t_mean_ratings ON t_mean_ratings.name_id = books.book_id
            LEFT JOIN(
                SELECT
                    COUNT(review_id) as cnt_review,
                    book_id as book_id_reviews
                FROM
                    reviews
                GROUP BY
                    book_id
            ) as t_cnt_review ON t_cnt_review.book_id_reviews = books.book_id
        GROUP BY
            book_id,
            t_mean_ratings.avg_rating,
            t_cnt_review.cnt_review
        ORDER BY
            "Оценка" DESC
        '''
pd.io.sql.read_sql(query, con=engine)

,ID книги,Название книги,Оценка,Количество обзоров
0,229,Evil Under the Sun (Hercule Poirot #24),5.00,2.0
1,55,A Woman of Substance (Emma Harte Saga #1),5.00,2.0
2,518,Pop Goes the Weasel (Alex Cross #5),5.00,2.0
3,902,The War of Art: Break Through the Blocks & Win...,5.00,2.0
4,421,Marvel 1602,5.00,2.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,371,Junky,2.00,2.0
997,316,His Excellency: George Washington,2.00,2.0
998,202,Drowning Ruth,2.00,3.0


## 4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры:

In [9]:
query = '''
    SELECT
        count_books as "Количество книг",
        publisher as "Издательство"
    FROM (
        SELECT
            COUNT(num_pages) as count_books,
            publisher_id
        FROM
            books
        WHERE
            num_pages > 50
        GROUP BY
            publisher_id) as sub
        LEFT JOIN publishers on sub.publisher_id = publishers.publisher_id
    WHERE
        count_books = (
            SELECT
                MAX(count_books) as max_cnt
            FROM (
                    SELECT
                        COUNT(num_pages) as count_books,
                        publisher_id
                    FROM
                        books
                    WHERE
                        num_pages > 50
                    GROUP BY
                        publisher_id) as sub2
        )
        '''
pd.io.sql.read_sql(query, con=engine)


,Количество книг,Издательство
0,42,Penguin Books


# 5. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [28]:
query = '''
    SELECT
        author_id as "ID автора",
        ROUND(AVG(max_rating.avg_rating), 3) as "Рейтинг",
        author
    FROM
        books
        RIGHT JOIN(
            SELECT
                author_id as a_id,
                author
            FROM
                authors
                ) as max_author on max_author.a_id = books.author_id
        RIGHT JOIN(
            SELECT
                book_id as b_id,
                COUNT(rating_id) as cnt_rating,
                AVG(rating) as avg_rating
            FROM
                ratings
            GROUP BY
                book_id
            HAVING
                COUNT(rating_id) >= 50
            ORDER BY 
                avg_rating DESC
                ) as max_rating on max_rating.b_id = books.book_id
    GROUP BY
        author_id,
        author
    ORDER BY 
        "Рейтинг" DESC
    LIMIT 1
        '''
pd.io.sql.read_sql(query, con=engine)

,ID автора,Рейтинг,author
0,236,4.284,J.K. Rowling/Mary GrandPré


# 6. Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''
    SELECT
        ROUND(AVG(cnt.cnt_r), 1) as "Среднее количество обзоров"
    FROM (
        SELECT
            COUNT(review_id) as "cnt_r"
        FROM 
            reviews
            RIGHT JOIN(
                SELECT
                    username as id,
                    COUNT(rating_id) as "cnt_rating"
                FROM
                    ratings
                GROUP BY
                    username
                HAVING
                    COUNT(rating_id) > 50
                    ) as count_rating on count_rating.id = reviews.username
        GROUP BY
            username
        ORDER BY 
            cnt_r DESC
        ) as cnt
        '''
pd.io.sql.read_sql(query, con=engine)

,Среднее количество обзоров
0,24.3
